In [ ]:
import numpy as np
import plotly.express as px
import cv2

In [ ]:
images = []
for size in [50, 100, 200]:
    img = np.load(f"data/photo_approx_{size}.npy")
    edge_frame = cv2.blur(img, (3, 3))
    canny = cv2.Canny(edge_frame, 10, 70, apertureSize=3)
    images.append(canny)

px.imshow(np.array(images), animation_frame=0)

# 14.3, 14.4 cm shape
